# CMR User Configurator

To use this system with its default settings, you will need an account from TACC. If you don't already have a TACC login, please go there now and sign up.

The simplest way to use the User Configurator for the Coastal Model Repository (CMR), is to execute the cells below, one at a time.

In [1]:
from setvar import *
import os, re, sys

In [2]:
# Some settings needed by the system
setvar("""
AGAVE_JSON_PARSER=jq
PATH=/usr/local/cli/bin:/usr/bin:/bin:/usr/sbin:/usr/local/sbin:/sbin
AGAVE_TENANTS_API_BASEURL=https://api.tacc.utexas.edu/tenants
""")

AGAVE_JSON_PARSER=jq
PATH=/usr/local/cli/bin:/usr/bin:/bin:/usr/sbin:/usr/local/sbin:/sbin
AGAVE_TENANTS_API_BASEURL=https://api.tacc.utexas.edu/tenants


## Step 1: Choose your tenant
The tenant is the system that manages your login credentials and Agave data. You
can choose any of the systems below. By default, this notebook uses tacc.prod.

In [3]:
# In case you don't want to use tacc.prod, here is
# a list of possible Agave tenants.
!tenants-list

ID                   NAME                                     URL                                               
araport.org          Araport                                  https://api.araport.org/                          
agave.prod           Agave Public Tenant                      https://public.agaveapi.co/                       
portals              Portals Tenant                           https://portals-api.tacc.utexas.edu/              
sgci                 Science Gateways Community Institute     https://sgci.tacc.cloud/                          
bridge               Bridge                                   https://api.bridge.tacc.cloud/                    
designsafe           DesignSafe                               https://agave.designsafe-ci.org/                  
sd2e                 SD2E Tenant                              https://api.sd2e.org/                             
tacc.prod            TACC                                     https://api.tacc.utexas.edu/      

#### Edit this next line to change the tenant.

In [4]:
# Set the tenant.
#!tenants-init -t tacc.prod

## Step 2: Supply your password

In [5]:
# Run the cell below to load your AGAVE_PASSWD
# (which will be your TACC password if you use
# the tacc.prod tenant, which is the default).
# By default, this will be the password stored
# in AGAVE_PASSWD.txt. If that file does not exist
# you will be prompted for your password.
#
# If, for some reason, that password is incorrect,
# you can get the system to prompt you again by
# uncommenting the next line and removing the file.
#os.remove("AGAVE_PASSWD.txt")
readpass("AGAVE_PASSWD")

Password or secret: AGAVE_PASSWD
Reading file `AGAVE_PASSWD.txt'


## Step 3: Supply your Username and Email

In [6]:
# Fill in your Agave username to the right of
# the eqauls sign. (This will be your TACC username
# if you are using the tacc.prod tenant, which is
# the default)
setvar("""
AGAVE_USERNAME=ysboss
EMAIL=syuan@cct.lsu.edu
""")

AGAVE_USERNAME=ysboss
EMAIL=syuan@cct.lsu.edu


## Step 4: Supply the name of the Agave Application you will be using.
You should obtain this from someone who as set up the CMR using the
ResourceConcifguator. By default, jobs will execute on shelob.hpc.lsu.edu
and you will need to contact sbrandt@cct.lsu.edu to gain permission to
run jobs.

## Step 5: Supply your PushBullet Token
Please go to https://www.pushbullet.com/ to obtain your token.

In [7]:
# Optional:
# The value of your PushBullet token goes here.
# As with the AGAVE_PASSWD entry above, please
# delete it if you wish to use PushBullet to
# receive notifications.
#
#os.remove("PBTOK.txt")
readpass("PBTOK")

Password or secret: PBTOK
Reading file `PBTOK.txt'


# END DATA ENTRY
If you have entered the data above correctly,
you should not need to edit anything below
this line. You will, however, need to execute
the cells below.

In [8]:
# Some commands to initialize your access to the system
setvar("CLIENT_NAME=user-${AGAVE_USERNAME}")
#!clients-delete -u $AGAVE_USERNAME -p $AGAVE_PASSWD $CLIENT_NAME
#!clients-create -p $AGAVE_PASSWD -S -N $CLIENT_NAME -u $AGAVE_USERNAME
#!auth-tokens-create -u $AGAVE_USERNAME -p $AGAVE_PASSWD
!auth-session-init -t tacc.prod -u $AGAVE_USERNAME -p $AGAVE_PASSWD -N $CLIENT_NAME

CLIENT_NAME=user-ysboss
Loading user-ysboss for ysboss from tacc.prod
Refreshing access token for user-ysboss...
30ef87dcc1242913df87e7ea2cf4aa6


In [9]:
# Obtain the list of systems
!auth-tokens-refresh
import systemdata
systemdata.display()

Refreshing access token...
30ef87dcc1242913df87e7ea2cf4aa6
New token expires: Sun Oct 6 23:43:47 CDT 2019
No handlers could be found for logger "agavepy.utils.save_configs"
cmd: apps-list ...
cmd: apps-list ...
cmd: apps-list ...
cmd: apps-list ...
cmd: apps-list ...
Applications you have access to:
  - crcollaboratory-tapis-shelob-ysboss-2.0
  - crcollaboratory-shelob-ysboss-2.0
  - crcollaboratory-shelob-tg457049-2.0
  - crcollaboratory-melete05-tg457049-2.0
  - crcollaboratory-spine-tg457049-2.0

Applications you don't have access to:
  None

Please contact sbrandt@cct.lsu.edu for questions


In [10]:
!apps-list

crcollaboratory-tapis-shelob-ysboss-2.0
crcollaboratory-shelob-ysboss-2.0
tapis.app.imageclassify-1.0u3
tapis.app.imageclassify-1.0u2
tapis.app.imageclassify-1.0u1
crcollaboratory-shelob-tg457049-2.0
crcollaboratory-melete05-tg457049-2.0
crcollaboratory-spine-tg457049-2.0
vina-ls5-1.1.2u3
vina-ls5-1.1.2u2
vina-ls5-1.1.2u1
opensees-2.4.4-slurm-2.4.4.5804u1
opensees-fork-2.4.4.5804u2
opensees-2.4.4.5804u1
opensees-fork-2.4.4.5804u1
vina-lonestar-1.1.2u4
vina-lonestar-1.1.2u3
vina-lonestar-1.1.2u2
pdb2pdbqt-1.00u1
